In [1]:
# %%%%%% Capstone %%%%%%
# %%%%%% Caroline Sklaver %%%%%%
# %%%%%% Date: June - 23 - 2020 %%%%%%
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %%%%%% Imputing with KNN %%%%%%
#%%-----------------------------------------------------------------------

#%%-----------------------------------------------------------------------
# Importing the required packages

import numpy as np
import pandas as pd


path = ('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/')

import os
os.chdir(path)

#%%-----------------------------------------------------------------------
# Importing the data

df_raw = pd.read_csv('df_raw_ext.csv')
df_raw.drop(['SEQN'], axis=1, inplace=True)
df_raw.drop(['year'],axis=1, inplace=True)
df_raw.drop(['cancer_type','mom_smoked_preg', 'days_60min_active','activity_week'],axis=1, inplace=True)




pd.options.mode.chained_assignment = None 
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
#continuous features
cont = ['#_ppl_household','age','annual_HI','caffeine','protein','carb','sugar',
        'fiber','fat','diet_chol','VitB6','folate','VitB12','magnesium','iron','zinc','selenium','water',
        'healthcare_past_yr','#_rooms','hrs_worked_week','lab_B12','hours_TV','hours_computer',
        'hrs_sleep','lifetime_partners','triglyceride','tot_cholesterol','glycohemoglobin','rbc_folate',
        'serum_folate','fasting_glucose','lab_VitD','systolic_BP','diastolic_BP','pulse','BMI','waist_C',
        'VitD','#meals_fast_food','monthly_income','min_sedetary','serum_selenium','serum_zinc','serum_creatinine',
        'lab_VitB12', 'bone_mineral_density','#_chronic_condition', 'hours_screen', 'combined_BP', 
        'people_per_rooms','diet_tot_VitB', 'age_2', '#_ppl_household_2', 'annual_HI_2',
       'caffeine_2', 'protein_2', 'carb_2', 'sugar_2', 'fiber_2', 'fat_2',
       'diet_chol_2', 'VitB6_2', 'folate_2', 'VitB12_2', 'magnesium_2',
       'iron_2', 'zinc_2', 'selenium_2', 'water_2', 'healthcare_past_yr_2',
       '#_rooms_2', 'hrs_worked_week_2', 'lab_B12_2', 'hours_TV_2',
       'hours_computer_2', 'hrs_sleep_2', 'lifetime_partners_2',
       'triglyceride_2', 'tot_cholesterol_2', 'glycohemoglobin_2',
       'rbc_folate_2', 'serum_folate_2', 'fasting_glucose_2', 'lab_VitD_2',
       'systolic_BP_2', 'diastolic_BP_2', 'pulse_2', 'BMI_2', 'waist_C_2',
       'VitD_2', '#meals_fast_food_2', 'monthly_income_2', 'min_sedetary_2',
       'serum_selenium_2', 'serum_zinc_2', 'serum_creatinine_2',
       'lab_VitB12_2', 'bone_mineral_density_2', '#_chronic_condition_2',
       'hours_screen_2', 'combined_BP_2', 'people_per_rooms_2']

non_cont = np.setdiff1d(df_raw.columns, [cont])


# cont_ss = []
# for var in cont:
#     if var in df_raw.columns:
#         cont_ss.append(var)

# df_raw[cont_ss] = ss.fit_transform(df_raw[cont_ss])

In [2]:
df_raw.head()

,depressed,broken_hip,doc_osteoporosis,race_ethnicity,edu_level,#_ppl_household,age,gender,annual_HI,marital_status,...,serum_selenium,serum_zinc,serum_creatinine,lab_VitB12,bone_mineral_density,difficult_hearing,difficult_seeing,difficult_walking,difficult_errands,doc_COPD
0,0.0,0.0,0.0,4.0,4.0,4.0,44.0,0.0,11.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,3.0,5.0,2.0,70.0,1.0,11.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,3.0,3.0,2.0,73.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,NaN,NaN,2.0,NaN,3.0,18.0,0.0,11.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,NaN,NaN,3.0,NaN,3.0,19.0,1.0,11.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_raw_og = pd.read_csv('df_raw_v2.csv')
df_raw_og.head()
df_raw_og.drop(['SEQN'], axis=1, inplace=True)

In [3]:
#%%-----------------------------------------------------------------------
# Function to identify missing values

def nan_helper(df):
    """
    The NaN helper

    Parameters
    ----------
    df : dataframe
    
    Returns
    ----------
    The dataframe of variables with NaN (index), 
    raw number missing, and their proportion
    
    """
    
    
    # get the raw number of missing values & sort
    missing = df.isnull().sum().sort_values(ascending=True)
    
    # get the proportion of missing values (%)
    proportion = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=True)
    
    # create table of missing data
    nan_data = pd.concat([missing, proportion], axis=1, keys=['missing', 'proportion'])
    
    return nan_data

In [4]:
nan_df = nan_helper(df_raw)
nan_df.head(7)

,missing,proportion
depressed,0,0.000000
race_ethnicity,0,0.000000
#_ppl_household,0,0.000000
age,0,0.000000
gender,0,0.000000
vigorous_activity,3,0.000096
moderate_activity,6,0.000191


In [5]:
def missing_values(df, threshold_col, threshold_row, impute_type):
    """
    Handle Missing Values

    Parameters
    ----------
    df : dataframe
    threshold_col: the proportion of missing values at which  to drop whole column
    threshold_row: the proportion of missing values at which to drop rows
    impute_type: mean or median imputation for continuous variables
    
    Returns
    ----------
    The dataframe without missing values
    
    """
    
    # Dropping Cols and Rows
    # call NaN helper function
    df_nan = nan_helper(df)
        
    # drop columns with higher proportion missing than threshold col
    df = df.drop((df_nan[df_nan['proportion'] > threshold_col]).index,1)
    
    # drop rows with higher proportion missing than threshold row
    df_nan_2 = df_nan[df_nan['proportion']>threshold_row]
    df = df.dropna(subset=np.intersect1d(df_nan_2.index, df.columns),
                           inplace=False)
    

    
    # Imputing values
    # Impute continuous variables with mean 
    if impute_type == 'mean':
        for col in cont:
            if col in df.columns:
                df[col].fillna(df[col].mean(), inplace=True)
    # Impute continuous variables with median
    elif impute_type == 'median':
        for col in cont:
            if col in df.columns:
                df[col].fillna(df[col].median(), inplace=True)
    
    
    # Impute categorical variables with most frequent/mode
    for col in non_cont:
        if col in df.columns:
            df[col].fillna(df[col].value_counts().index[0], inplace=True)
    

    return df

In [12]:
# example input
#df_new = missing_values(df_raw, 0.75, 0.75, "mean")

df_new_median = missing_values(df_raw, 0.75, 0.75, "median")

df_og_median = missing_values(df_raw_og, 0.75, 0.75, "median")
df_og_mean = missing_values(df_raw_og, 0.75, 0.75, "median")

In [13]:
df_og_mean.head()

,race_ethnicity,edu_level,#_ppl_household,age,gender,marital_status,annual_HI,depressed,caffeine,doc_diabetes,...,systolic_BP,diastolic_BP,BMI,waist_C,year,#meals_fast_food,min_sedetary,bone_mineral_density,difficult_hearing,doc_COPD
0,4.0,4.0,4.0,44.0,2.0,1.0,11.0,0.0,1.300000e+01,0.0,...,144.0,74.0,30.90,96.0,2005-2006,1.0,300.0,1.108,0.0,0.0
1,3.0,5.0,2.0,70.0,1.0,1.0,11.0,0.0,2.600000e+02,1.0,...,138.0,60.0,24.74,96.5,2005-2006,1.0,300.0,1.108,0.0,0.0
2,3.0,3.0,2.0,73.0,1.0,1.0,6.0,0.0,1.420000e+02,0.0,...,130.0,68.0,30.63,117.1,2005-2006,1.0,300.0,1.108,0.0,0.0
3,2.0,4.0,3.0,18.0,2.0,5.0,11.0,0.0,5.397605e-79,0.0,...,110.0,64.0,29.45,84.0,2005-2006,1.0,300.0,1.108,0.0,0.0
4,3.0,4.0,3.0,19.0,1.0,5.0,11.0,0.0,5.397605e-79,0.0,...,108.0,62.0,22.57,84.2,2005-2006,1.0,300.0,1.108,0.0,0.0


In [17]:
df_new_median.head()

,depressed,broken_hip,doc_osteoporosis,race_ethnicity,edu_level,#_ppl_household,age,gender,annual_HI,marital_status,...,savings_5000,min_sedetary,oral_rec,lab_VitB12,bone_mineral_density,difficult_hearing,difficult_seeing,difficult_walking,difficult_errands,doc_COPD
0,0.0,0.0,0.0,4.0,4.0,4.0,44.0,0.0,11.0,1.0,...,0.0,300.0,4.0,520.0,1.108,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,3.0,5.0,2.0,70.0,1.0,11.0,1.0,...,0.0,300.0,4.0,520.0,1.108,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,3.0,3.0,2.0,73.0,1.0,7.0,1.0,...,0.0,300.0,4.0,520.0,1.108,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,2.0,4.0,3.0,18.0,0.0,11.0,5.0,...,0.0,300.0,4.0,520.0,1.108,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,3.0,4.0,3.0,19.0,1.0,11.0,5.0,...,0.0,300.0,4.0,520.0,1.108,0.0,0.0,0.0,0.0,0.0


In [16]:
# df_new.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_mean_75_feature_ext.csv', index=False)

# export new median, old median and old mean
df_new_median.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_median_75_ext.csv', index=False)
#df_og_median.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_median_75_2.csv', index=False)
#df_og_mean.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_mean_75_2.csv', index=False)




In [19]:
# #continuous features
# cont = ['edu_level','#_ppl_household','age','annual_HI','caffeine','protein','carb','sugar',
#         'fiber','fat','diet_chol','VitB6','folate','VitB12','magnesium','iron','zinc','selenium','water',
#         'healthcare_past_yr','#_rooms','hrs_worked_week','lab_B12','hours_TV','hours_computer',
#         'hrs_sleep','lifetime_partners','triglyceride','tot_cholesterol','glycohemoglobin','rbc_folate',
#         'serum_folate','fasting_glucose','lab_VitD','systolic_BP','diastolic_BP','pulse','BMI','waist_C',
#         'VitD','#meals_fast_food','monthly_income','min_sedetary','serum_selenium','serum_zinc','serum_creatinine',
#         'lab_VitB12', 'bone_mineral_density']

# # categorical features
# binary = ['broken_hip', 'doc_osteoporosis', 'gender', 'special_diet',
#        'doc_diabetes', 'used_CMH', 'health_insurance', 'mom_smoked_preg',
#        'adults_skip_meals', 'food_stamps', 'WIC', 'doc_asthma',
#        'doc_overweight', 'doc_arthritis', 'doc_CHF', 'doc_CHD',
#        'doc_heart_attack', 'doc_stroke', 'doc_chronic_bronchitis',
#        'doc_liver_condition', 'doc_thyroid_problem', 'doc_cancer', 'anemia',
#        'doc_kidney', 'vigorous_activity', 'moderate_activity', 'limit_work',
#        'PME_limit', 'doc_sleeping_disorder', 'smoker', 'smoked_5days',
#        'alcoholic', 'tried_lose_weight', 'herpes_2', 'HIV', 'savings_5000',
#        'days_60min_active', 'doc_HPV', 'difficult_remembering', 'prob_smell',
#        'prob_tatse', 'persistent_cold', 'nasal_congestion',
#        'difficult_hearing', 'difficult_seeing', 'difficult_walking',
#        'difficult_errands', 'doc_COPD', 'activity_week']

# multi-class features
cat_encode = ['manage_$_difficult', 'gen_health_cond', 'consider_weight',
              'food_didnt_last', 'how_healthy_diet', 'place_healthcare',
              'work_type', 'embarassed_mouth', 'race_ethnicity', 'race_cat',
              'home_ownership', 'food_security_cat','edu_level','embarassed_mouth',
              'relied_cheap_food', 'marital_status', 'afford_balanced_meals',
              'run_out_food', 'oral_rec', 'sexual_orientation','afford_balanced_meals', 
              'age_cat', 'food']

In [ ]:
from sklearn.model_selection import train_test_split

# # Function to One-hot-encode the categorical features
# def one_hot_encode(df):
#     cols = df.columns
#     e_cols = list(set(cols).intersection(set(cat_encode)))
#     df_encode = pd.get_dummies(df, columns=e_cols)

#     return (df_encode)

# df = one_hot_encode(df_raw)

# df_raw['age_cat'] = pd.get_dummies(df_raw, columns=['age_cat'])

# get the name of the features
#features = np.setdiff1d(df.columns, [target])

In [30]:
#%%-------------------------------------------------------------------------

# # KNN Progressive Model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor(n_neighbors=5)
knc = KNeighborsClassifier(n_neighbors=5)

# MLP Model
from sklearn.neural_network import MLPClassifier, MLPRegressor
mlpc = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42)
mlpr = MLPRegressor(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42)

def impute_progressive_knn(df, nan_df):

    predictions = []
    target = None
    df_copy=df.copy()

    for i in range(6,len(nan_df)+1):
        # handling identifiers
        df_start = df_copy[nan_df.iloc[1:i, ].index]

        # get the target name
        target = df_start.columns[-1]
        #print(target)


        # get the name of the features
        features = np.setdiff1d(df_start.columns, [target])

        # Get NaN rows
        null_data = df_start[df_start.isnull().any(axis=1)]

        # drop target from NaN rows to get what rows we want to predict with
        imp = null_data[features].to_numpy()

        # drop Na rows from df
        df_1 = df_start.dropna()

        # split into training and target
        X_train = df_1[features].to_numpy()
        y_train = df_1[target].astype(int).to_numpy()

        # fit the model
        if target in cont:
            predictions = knr.fit(X_train, y_train).predict(imp)
        else:
            predictions = knc.fit(X_train, y_train).predict(imp)

        # find the values and indices to impute
        fill = pd.DataFrame(index=df_start.index[df_start.isnull().any(axis=1)],
                             data=predictions, columns=[target])

        # fill the df with predictions
        df_copy.fillna(fill, inplace=True)


    return df_copy

In [31]:
df_progressive_knn = impute_progressive_knn(df_raw, nan_df)

In [32]:
df_progressive_knn

,depressed,broken_hip,doc_osteoporosis,race_ethnicity,edu_level,#_ppl_household,age,gender,annual_HI,marital_status,...,serum_selenium,serum_zinc,serum_creatinine,lab_VitB12,bone_mineral_density,difficult_hearing,difficult_seeing,difficult_walking,difficult_errands,doc_COPD
0,0.0,0.0,0.0,4.0,4.0,4.0,44.0,0.0,11.0,1.0,...,113.2,73.0,188.2,415.6,0.800,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,3.0,5.0,2.0,70.0,1.0,11.0,1.0,...,121.0,81.4,94.2,266.4,0.800,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,3.0,3.0,2.0,73.0,1.0,5.8,1.0,...,129.6,78.8,138.4,549.0,1.000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,2.0,5.0,3.0,18.0,0.0,11.0,5.0,...,130.4,87.6,66.0,486.2,0.800,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,3.0,5.0,3.0,19.0,1.0,11.0,5.0,...,112.8,82.6,146.2,547.6,1.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31352,0.0,0.0,0.0,5.0,5.0,7.0,25.0,1.0,10.0,5.0,...,124.6,68.5,100.8,545.2,0.733,0.0,0.0,0.0,0.0,0.0
31353,0.0,0.0,0.0,3.0,3.0,1.0,76.0,0.0,4.0,2.0,...,111.8,78.8,52.6,723.8,0.800,0.0,0.0,1.0,0.0,0.0
31354,0.0,0.0,0.0,3.0,5.0,3.0,26.0,1.0,9.0,1.0,...,121.0,73.0,124.8,578.2,1.226,0.0,0.0,0.0,0.0,0.0
31355,0.0,0.0,0.0,3.0,4.0,1.0,80.0,0.0,7.0,2.0,...,141.0,83.8,91.6,1233.8,0.600,1.0,1.0,1.0,1.0,0.0


In [33]:
#print(df_progressive_knn.head())

df_progressive_knn.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_progressive_knn_ext_ext.csv', index=False)





In [28]:
df_progressive_mlp.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_progressive_mlp_ext_ext.csv', index=False)

In [ ]:
df_progressive_mlp